# Food in Art

### angles

- Proportion of food related items in museums
- Proportion of food related artworks by epoch and country
- Is food central topic
- Network diagram linking culinary styles with corresponding art movements.
- fruit, meal, dessert, etc
- feasts, banquets, etc
- food as a medium?
- food symbolism?
- **Analysis Angle:** Investigate how the introduction of new food items through trade influenced their depiction in art.
    - **Visualization:** Maps showing trade routes alongside corresponding changes in food representation in art."
- compare current food focus by country
- Nutrional evolution
- **Analysis Angle:** Explore how periods of famine or abundance influenced the portrayal of food in art.
    - **Visualization:** Correlation charts linking historical events of scarcity or abundance with changes in food depiction."
- Social classes?
- Agriculture, markets?
- **Analysis Angle:** Analyze the use of color in food-themed art and its psychological effects on viewers.
- **Visualization:** Color wheel charts displaying dominant hues in various artworks."
- **Analysis Angle:** Investigate how chefs create dishes inspired by artworks, merging visual art with gastronomy.
- **Visualization:** Side-by-side images of artworks and the culinary dishes they inspired."

In [ ]:
print("let's go")

In [23]:
import pywikibot
from pywikibot import pagegenerators
import xml.etree.ElementTree as ET
import pandas as pd
from lxml import etree
import re
from urllib.parse import quote

## TODO
1. Set columns : AUTHOR TITLE DATE ORIGIN_COUNTRY DISPLAY_COUNTRY DISPLAY_LOCATION TYPE SCHOOL TIMEFRAME? TIME_PERIOD  WA_URL WIKI_URL WIKIDATA_ID IMAGE_URL KEYWORDS
2. Get data from web gallery
3. Clean and filter data, only paintings, remove dupes, remove (details) cf mona lisa
4. Scrape web gallery for description and image url
5. Save to CSV, cache
6. fetch all images
5. Get data from wikidata
7. supplement first df
6. remove dupes
4. fine tune yolo5 for food recognition using wikidata

In [ ]:
#TODO caching system

In [25]:

def parse_wikipedia_abstracts_lxml(xml_file, words_in_abstract):
    data = []
    context = etree.iterparse(xml_file, events=('end',), tag='doc')
    for event, elem in context:
        url = elem.findtext('url')
        title = elem.findtext('title')
        abstract = elem.findtext('abstract')
        if abstract:
            for word in words_in_abstract:
                # Check for exact word match and its plural form
                if re.search(r'\b' + re.escape(word) + r's?\b', abstract):
                    data.append({
                        'URL': url,
                        'Title': title,
                        'Abstract': abstract,
                        'Keyword': word
                    })
                    break
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    return pd.DataFrame(data, columns=['URL', 'Title', 'Abstract', 'Keyword'])


In [ ]:
# Path to your enwiki-latest-abstract.xml file
xml_file_path = 'data/enwiki-latest-abstract.xml'

words_in_abstract = ['painting', 'gravure', 'illustration', 'mural', 'fresco', 'watercolor', 'pastel', 'etching', 'lithography', 'woodcut', 'silkscreen']

# Parse the XML and create the DataFrame
wiki_df = parse_wikipedia_abstracts_lxml(xml_file_path, words_in_abstract)

# Display the first few rows
print(wiki_df.head())

In [27]:
# Save to CSV
wiki_df.to_csv('data/wikipedia_abstracts.csv', index=False)